# 【AAPlot for Animal Behavior (Pretreat)】
## Extract speed and trajectory of EzTrack exported .csv file

Run under `PLOT` environment
    
The `PLOT` enviorment：
- Python3.12.7
- pandas
- numpy
- matplotlib
- seaborn
- ipykernel

*Warning*

*! Make sure you have installed `Anaconda`，and added to PATH（refering to internet）*

*! Make sure you have already confiured the `PLOT` environment, if not, run this command: `conda env create -n PLOT python=3.12.7 pandas numpy matplotlib seaborn ipykernel` (If you are using ARM64 CPU, use Python3.13.3 intead，add `conda-forge` at the end of command)*

*Apply `PLOT` in VScode ：Select in the Kernel*

*Apply `PLOT` in terminal：`conda activate PLOT`*

### Step1. Data Input

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Filename = r'Z:\Ruyi_RawData\Fiber\eCB\DrugIntake\Behavior\NAcSh\Saline\02 Processed\C15_LocationOutput.csv'
# Filename = 'Test.csv' # for Linux/MacOS

# Read the CSV file
df = pd.read_csv(Filename)

# Create a conversion factor from pixels to cm using the first row's data
px_distance = df.loc[1, 'Distance_px']  # Get the distance in pixels
cm_distance = df.loc[1, 'Distance_cm']  # Get the distance in cm
conversion_factor = cm_distance / px_distance  # Calculate conversion factor (cm/px)

# Extract data and convert to appropriate units
frame_numbers = df['Frame']
time_seconds = frame_numbers / 30  # Convert frames to seconds (30Hz)
x_trajectory_cm = df['X'] * conversion_factor  # Convert X to cm
y_trajectory_cm = df['Y'] * conversion_factor  # Convert Y to cm
distance_cm = df['Distance_cm']  # Already in cm

# Calculate speed in cm/s (multiply by frame rate since distance_cm is per frame)
speed_cm_s = distance_cm * 30  # Convert cm/frame to cm/s

# Calculate cumulative distance
cumulative_distance_cm = distance_cm.cumsum()

# Create a summary dataframe with only cm data
summary_df = pd.DataFrame({
    'Time_seconds': time_seconds,
    'X_cm': x_trajectory_cm,
    'Y_cm': y_trajectory_cm,
    'Distance_cm': distance_cm,
    'Speed_cm_s': speed_cm_s,
})

print("Data Summary:")
print(summary_df.head())
print("\nNote: All distances in centimeters, time in seconds, speed in cm/s")

Data Summary:
   Time_seconds       X_cm       Y_cm  Distance_cm  Speed_cm_s
0      0.000000  49.788406  46.904752     0.000000    0.000000
1      0.033333  34.734721  52.544160    16.075334  482.260013
2      0.066667  60.222828  36.573165    30.078502  902.355046
3      0.100000  60.353209  35.826073     0.758384   22.751508
4      0.133333  60.419533  34.221993     1.605450   48.163506

Note: All distances in centimeters, time in seconds, speed in cm/s


### 2. Plot trajectory and Time-Speed

In [20]:
# Get input filename without extension
base_name = os.path.splitext(Filename)[0]

# 1. Trajectory Plot (simplified version)
fig, ax = plt.subplots(figsize=(4.5, 4.5))  # Reduced by 40% from 7.5x7.5
ax.plot(x_trajectory_cm, y_trajectory_cm, 'r-', linewidth=1.5)  # Red trajectory line

# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('X Position (cm)', fontsize=12)
ax.set_ylabel('Y Position (cm)', fontsize=12)
ax.axis('equal')  # Make sure x and y scales are equal
ax.grid(False)  # Remove grid

plt.tight_layout()
trajectory_filename = f"{base_name}_Trajectory.svg"
plt.savefig(trajectory_filename, format='svg', bbox_inches='tight', dpi=300)
plt.close()  # Close the figure to free memory

# 2. Time-Speed Plot (with Savitzky-Golay filter)
fig, ax = plt.subplots(figsize=(6, 2.5))  # Width reduced by 50%, height by 40%

# Apply Savitzky-Golay filter
# Adjust window length based on data size
from scipy.signal import savgol_filter
data_length = len(speed_cm_s)
window_length = min(31, data_length)  # About 1 second, ensuring it's less than data length
if window_length % 2 == 0:  # Make window length is odd
    window_length -= 1
polyorder = 2  # Lower polynomial order for smoother result
smooth_speed = savgol_filter(speed_cm_s, window_length=window_length, polyorder=polyorder)

# Plot smoothed speed
ax.plot(time_seconds, smooth_speed, 'r-', linewidth=2, label='Smoothed Speed')

# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Speed (cm/s)', fontsize=12)
ax.legend(fontsize=10)
ax.grid(False)  # Remove grid

plt.tight_layout()
speed_plot_filename = f"{base_name}_TimeOverSpeed.svg"
plt.savefig(speed_plot_filename, format='svg', bbox_inches='tight', dpi=300)
plt.close()  # Close the figure to free memory

# Calculate average distance per frame
avg_distance = distance_cm.mean()

# Print distance statistics
print(f"Total distance traveled: {cumulative_distance_cm.iloc[-1]:.2f} cm")
print(f"Average distance per frame: {avg_distance:.4f} cm")

# Create and save a CSV file with time and smoothed speed data
time_speed_df = pd.DataFrame({
    'Time(s)': time_seconds,
    'Speed(cm/s)': smooth_speed
})

# Save to CSV with input filename
speed_data_filename = f"{base_name}_TimeOverSpeed.csv"
time_speed_df.to_csv(speed_data_filename, index=False)
print(f"\nFiles saved:")
print(f"- {trajectory_filename}")
print(f"- {speed_plot_filename}")
print(f"- {speed_data_filename}")

Total distance traveled: 180036.71 cm
Average distance per frame: 0.3351 cm

Files saved:
- Z:\Ruyi_RawData\Fiber\eCB\DrugIntake\Behavior\NAcSh\Saline\02 Processed\C15_LocationOutput_Trajectory.svg
- Z:\Ruyi_RawData\Fiber\eCB\DrugIntake\Behavior\NAcSh\Saline\02 Processed\C15_LocationOutput_TimeOverSpeed.svg
- Z:\Ruyi_RawData\Fiber\eCB\DrugIntake\Behavior\NAcSh\Saline\02 Processed\C15_LocationOutput_TimeOverSpeed.csv

Files saved:
- Z:\Ruyi_RawData\Fiber\eCB\DrugIntake\Behavior\NAcSh\Saline\02 Processed\C15_LocationOutput_Trajectory.svg
- Z:\Ruyi_RawData\Fiber\eCB\DrugIntake\Behavior\NAcSh\Saline\02 Processed\C15_LocationOutput_TimeOverSpeed.svg
- Z:\Ruyi_RawData\Fiber\eCB\DrugIntake\Behavior\NAcSh\Saline\02 Processed\C15_LocationOutput_TimeOverSpeed.csv


### 3. CSV file output